In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import pandas_datareader as pdr
import pytictoc as tt
import plotly
from tickers500 import tickers500
from tickerTA import Ticker
from tickerTA import TechnicalAnalysis
from tradingStrategy1 import TradingStrategy1

# This is to avoid the SettingWithCopyWarning in pandas
pd.options.mode.copy_on_write = True 


In [9]:
# This class is used to parse the daily data from the Yahoo Finance API
# to implement the Ticker class, TechnicalAnalysis class and TradingStrategy1 class
# After processing all the data, it will output the current Buy/Sell recommendations list
# for the 500 S&P companies
# Also it will include the price change since the last recommendation

class Recommendations:
    def __init__(self, start = pd.to_datetime('today') - pd.DateOffset(months=12), end = pd.to_datetime('today')):
        self.start = start
        self.end = end
        self.tickers500 = tickers500
        self.buy_recommendations_df = pd.DataFrame()
        self.sell_recommendations_df = pd.DataFrame()
        self.get_stock_data()
        self.current_recommendations()

    def get_stock_data(self):
        self.tradingstrategy1_df = pd.DataFrame()
        self.signals_df = pd.DataFrame()
        for ticker in self.tickers500:
            stock = Ticker(ticker, self.start, self.end)
            # stock_df = stock_df._append(stock.df)
            techA = TechnicalAnalysis(stock)
            tradingS = TradingStrategy1(techA)
            self.signals_df = self.signals_df._append(tradingS.all_signals_df)
            self.tradingstrategy1_df = self.tradingstrategy1_df._append(tradingS.trades_df)
        self.signals_df = self.signals_df.sort_values(by='Date')

    def current_recommendations(self):
        for _, row in self.signals_df.iterrows():
            if row['Buy/Sell'] == 'Buy':
                self.buy_recommendations_df = self.buy_recommendations_df._append(row)
            elif row['Buy/Sell'] == 'Sell':
                self.sell_recommendations_df = self.sell_recommendations_df._append(row)
        self.buy_recommendations_df = self.buy_recommendations_df.sort_values(by='Date', ascending=False)
        self.sell_recommendations_df = self.sell_recommendations_df.sort_values(by='Date', ascending=False)
        # self.buy_recommendations_df = self.buy_recommendations_df.drop_duplicates(subset=['Ticker'])
        # self.sell_recommendations_df = self.sell_recommendations_df.drop_duplicates(subset=['Ticker'])

In [10]:
rec = Recommendations()


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FOX?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=FOX&crumb=dhI8XUMVh5p
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NWS?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=NWS&crumb=dhI8XUMVh5p
c:\Users\chris\Documents\Unreal Projects\AlgorithmicTrading\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1057: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\chris\Documents\Unreal Projects\AlgorithmicTrading\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1081: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [11]:

print(rec.signals_df)

   Ticker Buy/Sell                 Date   Price
0     CAG      Buy  2023-05-19 00:00:00   35.76
0      NI      Buy  2023-05-19 00:00:00   27.49
0    PODD      Buy  2023-05-19 00:00:00  302.97
0       D      Buy  2023-05-19 00:00:00   53.02
0      ED      Buy  2023-05-19 00:00:00   95.85
..    ...      ...                  ...     ...
5    TTWO     Sell  2024-03-22 00:00:00  152.67
3     MCO     Sell  2024-03-22 00:00:00  397.52
7    LULU     Sell  2024-03-22 00:00:00  416.25
2    CTSH      Buy  2024-03-22 00:00:00   72.61
2     ACN      Buy  2024-03-22 00:00:00  350.33

[2078 rows x 4 columns]


In [12]:
rec.buy_recommendations_df.to_csv('CSV/buy_recommendations.csv')
rec.buy_recommendations_df


,Ticker,Buy/Sell,Date,Price
2,ACN,Buy,2024-03-22,350.33
2,CTSH,Buy,2024-03-22,72.61
4,DRI,Buy,2024-03-22,164.10
2,ULTA,Buy,2024-03-21,523.96
4,EQIX,Buy,2024-03-21,825.00
...,...,...,...,...
0,PCG,Buy,2023-05-19,16.62
0,O,Buy,2023-05-19,60.31
0,AEP,Buy,2023-05-19,86.74
0,CNP,Buy,2023-05-19,28.59


In [13]:
rec.sell_recommendations_df.to_csv('CSV/sell_recommendations.csv')
rec.sell_recommendations_df


,Ticker,Buy/Sell,Date,Price
7,LULU,Sell,2024-03-22,416.25
3,MCO,Sell,2024-03-22,397.52
5,TTWO,Sell,2024-03-22,152.67
3,BK,Sell,2024-03-22,57.29
3,STT,Sell,2024-03-22,76.07
...,...,...,...,...
1,RHI,Sell,2023-06-07,71.89
1,ELV,Sell,2023-06-06,480.38
1,NDAQ,Sell,2023-06-05,56.25
1,CAT,Sell,2023-06-05,226.99


In [14]:
rec.tradingstrategy1_df.to_csv('CSV/tradingstrategy1.csv')
rec.tradingstrategy1_df

,Ticker,Buy Date,Buy Price,Buy P/E Ratio,Buy Fwd P/E Ratio,Buy P/B Ratio,Buy RSI,Buy Upper BB,Buy Bollinger %b,Buy Lower BB,...,Sell Awesome Oscillator,Sell Ultimate Oscillator,Sell TSI,Sell Acum/Dist,Profit,Profit %,Duration,Ticker Cum Profit,Ticker Cum Profit %,Profitable
0,MMM,2023-06-01,93.47,0.000000,0.000000,0.000000,30.292562,98.341894,0.030710,89.587106,...,4.940000,17.920265,23.298965,-1.627919e+09,16.37,17.51,55,16.37,17.51,Yes
1,MMM,2023-09-26,94.36,0.000000,0.000000,0.000000,11.622927,107.943333,-0.004423,90.776667,...,1.085529,10.755698,-5.067696,-2.664336e+09,-0.53,-0.56,41,15.84,16.95,No
2,MMM,2024-01-24,96.64,0.000000,0.000000,0.000000,10.744012,113.718373,-0.312097,96.685627,...,-0.162882,53.662246,-10.544182,-3.185987e+09,-2.66,-2.75,47,13.18,14.10,No
0,AOS,2023-06-01,64.17,23.948510,20.408777,7.068469,27.834573,70.267760,0.024639,63.512240,...,4.062324,27.267543,31.785812,-1.175580e+08,12.39,19.31,48,12.39,19.31,Yes
1,AOS,2023-08-21,68.87,23.948510,20.408777,7.068469,11.277113,74.791816,0.017214,68.072184,...,-0.297029,35.764834,-3.421279,-1.815260e+08,0.39,0.57,51,12.78,19.92,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,ZBRA,2024-02-01,239.55,52.240807,21.185726,5.048147,35.624809,263.373686,0.127750,239.297314,...,10.391324,58.610041,10.108574,5.892913e+06,42.86,17.89,15,18.59,7.43,Yes
0,ZBH,2023-05-24,129.82,26.211065,14.736175,2.107145,14.142161,142.561785,-0.141714,130.174215,...,3.571059,67.443670,16.857257,7.386438e+06,12.10,9.32,22,12.10,9.32,Yes
1,ZBH,2023-08-01,131.08,26.211065,14.736175,2.107145,13.322175,145.763365,-0.412053,136.048635,...,1.228441,55.586276,-1.829679,-1.202951e+07,-20.32,-15.50,107,-8.22,-6.33,No
0,ZTS,2023-05-25,169.63,33.067060,25.871912,15.377912,9.506281,190.474067,-0.155255,168.312933,...,5.442971,68.846333,17.960968,1.729535e+07,13.57,8.00,60,13.57,8.00,Yes
